In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.transforms import ToTensor, Compose, Grayscale, Resize, CenterCrop
import matplotlib.pyplot as plt
from PIL import Image
from parse import parse
from custom_loader import AgeDBDataset
from custom_loss_functions import AngularPenaltySMLoss

import warnings
warnings.filterwarnings("ignore")

In [2]:
# hyper params
num_of_class = 79
hidden_unit = 256
learning_rate = 1e-04
batch_size = 64
device = torch.device("cuda")

In [3]:
dataset = AgeDBDataset(
    directory = 'cropped_face/',
    transform = Compose([
        Resize(size=(64, 64)),
        CenterCrop(size=64),
        Grayscale(num_output_channels=1),
        ToTensor(),
    ]),
    device = device,
)

In [4]:
len(dataset)

15508

In [5]:
train_set, validation_set, test_set = dataset.get_loaders(
    batch_size=batch_size,
    train_size=0.8,
    test_size=0.2,
)

In [6]:
class AgeDBConvModel(nn.Module):
    def __init__(self, num_of_classes):
        super(AgeDBConvModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=2), #(64+2(2)-3)+1=66
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #((66-2)/2)+1 = 33
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=2), #(33+2(2)-3)+1=35
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #((35-2)/2)+1 = 17 + 0.5
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=2), #(17+2(2)-3)+1=19
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #((19-2)/2)+1 = 9 + 0.5
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=2), #(9+2(2)-3)+1=11
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #((11-2)/2)+1 = 5 + 0.5
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=2), #(5+2(2)-3)+1=7
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) #((7-2)/2)+1 = 3 + 0.5
        )
        self.fc1 = nn.Linear(3*3*512, 1*1*512)
        self.fc2 = nn.Linear(1*1*512, num_of_classes)
        self.smax = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)

        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.smax(out)
        
        return out

In [7]:
convModel1 = AgeDBConvModel(num_of_class).to(device)
convModel2 = AgeDBConvModel(num_of_class).to(device)
convModel3 = AgeDBConvModel(num_of_class).to(device)

In [8]:
# Training loop
def train(model, optimizer, criterion, train_loader, num_of_epoch):
    total_step = len(train_loader)
    for epoch in range(num_of_epoch):
        for i, (imgs, labels) in enumerate(train_loader):
            imgs = imgs.to(device)
            labels = torch.as_tensor(labels['age']).to(device)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1)%total_step == 0:
                print(f"Epoch: {epoch+1}/{num_of_epoch}, Step: {i+1}/{total_step}, Loss: {loss.item()}")

# Evaluation
def eval(model, test_loader):
    with torch.no_grad():
        correct = 0
        total = 0
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = torch.as_tensor(labels['age']).to(device)
            outputs = model(imgs)
            
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            
    print(f"Accuracy: {(100*correct)/total}%")

In [9]:
criteria = AngularPenaltySMLoss(
    in_features=num_of_class,
    out_features=num_of_class,
    loss_type='arcface',
)
optimizer = torch.optim.Adam(convModel1.parameters(), lr=learning_rate)
train(convModel1, optimizer, criteria, train_set, num_of_epoch=10)

Epoch: 1/10, Step: 194/194, Loss: 38.96859359741211
Epoch: 2/10, Step: 194/194, Loss: 39.296905517578125
Epoch: 3/10, Step: 194/194, Loss: 39.29692077636719
Epoch: 4/10, Step: 194/194, Loss: 39.29692840576172
Epoch: 5/10, Step: 194/194, Loss: 39.296939849853516
Epoch: 6/10, Step: 194/194, Loss: 39.296939849853516
Epoch: 7/10, Step: 194/194, Loss: 39.296939849853516
Epoch: 8/10, Step: 194/194, Loss: 39.29692840576172
Epoch: 9/10, Step: 194/194, Loss: 39.296913146972656
Epoch: 10/10, Step: 194/194, Loss: 39.29680252075195


In [10]:
eval(convModel1, test_set)
eval(convModel1, train_set)

Accuracy: 0.5482102547565302%
Accuracy: 0.7173948089634048%


In [11]:
criteria = AngularPenaltySMLoss(
    in_features=num_of_class,
    out_features=num_of_class,
    loss_type='cosface',
)
optimizer = torch.optim.Adam(convModel2.parameters(), lr=learning_rate)
train(convModel2, optimizer, criteria, train_set, num_of_epoch=10)

Epoch: 1/10, Step: 194/194, Loss: 16.62502098083496
Epoch: 2/10, Step: 194/194, Loss: 16.10080337524414
Epoch: 3/10, Step: 194/194, Loss: 15.949949264526367
Epoch: 4/10, Step: 194/194, Loss: 15.800993919372559
Epoch: 5/10, Step: 194/194, Loss: 15.589404106140137
Epoch: 6/10, Step: 194/194, Loss: 15.220233917236328
Epoch: 7/10, Step: 194/194, Loss: 14.723420143127441
Epoch: 8/10, Step: 194/194, Loss: 14.426335334777832
Epoch: 9/10, Step: 194/194, Loss: 14.143686294555664
Epoch: 10/10, Step: 194/194, Loss: 13.873109817504883


In [12]:
eval(convModel2, test_set)
eval(convModel2, train_set)

Accuracy: 1.4833924540470815%
Accuracy: 1.7088505561824923%


In [13]:
criteria = AngularPenaltySMLoss(
    in_features=num_of_class,
    out_features=num_of_class,
    loss_type='sphereface',
)
optimizer = torch.optim.Adam(convModel3.parameters(), lr=learning_rate)
train(convModel3, optimizer, criteria, train_set, num_of_epoch=10)

Epoch: 1/10, Step: 194/194, Loss: 40.65583419799805
Epoch: 2/10, Step: 194/194, Loss: 39.44287109375
Epoch: 3/10, Step: 194/194, Loss: 37.80892562866211
Epoch: 4/10, Step: 194/194, Loss: 37.693763732910156
Epoch: 5/10, Step: 194/194, Loss: 37.583770751953125
Epoch: 6/10, Step: 194/194, Loss: 37.41691970825195
Epoch: 7/10, Step: 194/194, Loss: 37.242156982421875
Epoch: 8/10, Step: 194/194, Loss: 36.93334197998047
Epoch: 9/10, Step: 194/194, Loss: 36.45634078979492
Epoch: 10/10, Step: 194/194, Loss: 35.54274368286133


In [14]:
eval(convModel3, test_set)
eval(convModel3, train_set)

Accuracy: 1.2254111576910673%
Accuracy: 0.9430920522327906%
